In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot

In [3]:
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [4]:
from datasets import load_dataset

dsd = load_dataset('bdsaglam/musique', 'answerable')
dsd

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
        num_rows: 19938
    })
    validation: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
        num_rows: 2417
    })
})

In [5]:
def make_example(record):
    decomposition = '\n'.join([f"{i+1}. {item['question']}" for i, item in enumerate(record["question_decomposition"])])
    return dspy.Example(
        question=record["question"],
        decomposition=decomposition,
    ).with_inputs("question")

In [6]:
make_example(dsd['train'][0])

Example({'question': 'When was the institute that owned The Collegian founded?', 'decomposition': '1. The Collegian >> owned by\n2. When was #1 founded?'}) (input_keys={'question'})

In [7]:
from itertools import islice

trainset = [make_example(record) for record in islice(iter(dsd['train']), 100)]
evalset = [make_example(record) for record in islice(iter(dsd['validation']), 100)]

In [8]:
dict(trainset[0])

{'question': 'When was the institute that owned The Collegian founded?',
 'decomposition': '1. The Collegian >> owned by\n2. When was #1 founded?'}

In [9]:
# Update the Signature
class DecomposeQuestion(dspy.Signature):
    """Decompose a complex question into simpler sub-questions."""

    question: str = dspy.InputField()
    decomposition: str = dspy.OutputField(desc="Enumerated list of sub-questions, using '#n >>' notation for dependent questions")


# Update the Module
class QuestionDecompositionModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.decompose = dspy.ChainOfThought(DecomposeQuestion)

    def forward(self, question):
        pred = self.decompose(question=question)
        return dspy.Prediction(decomposition=pred.decomposition)

def split_subquestions(decomposition_str):
    for line in decomposition_str.split("\n"):
        if line.strip():
            parts = line.split(". ", 1)
            if len(parts) == 1:
                return parts[0].strip
            elif len(parts) == 2:
                yield parts[1].strip()
            else:
                raise ValueError(f"Invalid decomposition line: {line}")


# Update the evaluation function
def evaluate_decomposition(example, pred, trace=None):
    gold_sub_questions = list(split_subquestions(example.decomposition))
    pred_sub_questions = list(split_subquestions(pred.decomposition))
    
    assert len(gold_sub_questions), "Gold decomposition is empty."

    exact_matches = len([1 for gold, pred in zip(gold_sub_questions, pred_sub_questions) if gold == pred])
    accuracy = exact_matches / len(gold_sub_questions)
    return accuracy

In [10]:
# ## 6. Implement the optimization process
# Initialize the uncompiled question decomposition module
uncompiled_qd = QuestionDecompositionModule()

# Set up the teleprompter
teleprompter = BootstrapFewShot(metric=evaluate_decomposition)

# Compile and optimize the question decomposition module
compiled_qd = teleprompter.compile(uncompiled_qd, trainset=trainset)

print("Question Decomposition module compiled and optimized")

  0%|          | 0/100 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'
  1%|          | 1/100 [00:01<02:48,  1.70s/it]Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'
  2%|▏         | 2/100 [00:03<02:40,  1.64s/it]Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'
  3%|▎         | 3/100 [00:04<02:38,  1.64s/it]Traceback 

Bootstrapped 4 full traces after 23 examples in round 0.
Question Decomposition module compiled and optimized


In [11]:
# Set up the evaluation function
evaluate_qd = Evaluate(devset=evalset, metric=evaluate_decomposition, num_threads=1, display_progress=True)

# Evaluate the uncompiled question decomposition module
uncompiled_score = evaluate_qd(uncompiled_qd)
print(f"Uncompiled Question Decomposition Module Score: {uncompiled_score}")

# Evaluate the compiled question decomposition module
compiled_score = evaluate_qd(compiled_qd)
print(f"Compiled Question Decomposition Module Score: {compiled_score}")


# ## 8. (Optional) Error Analysis


def perform_error_analysis(qd_module, dataset, num_samples=5):
    errors = []
    for example in dataset:
        pred = qd_module(question=example.question)
        if not evaluate_decomposition(example, pred):
            errors.append((example, pred))
    
    print(f"Total errors: {len(errors)}")
    print("\nSample of errors:")
    for example, pred in errors[:num_samples]:
        print(f"Original Question: {example.question}")
        print(f"# True Decomposition\n{example.decomposition}")
        print(f"# Predicted Decomposition\n{pred.decomposition}")
        print()

print("Error analysis for uncompiled Question Decomposition module:")
perform_error_analysis(uncompiled_qd, evalset)

print("\nError analysis for compiled Question Decomposition module:")
perform_error_analysis(compiled_qd, evalset)


compiled_qd.predictors()[0]


compiled_qd.save('compiled-qd.json')

Average Metric: 0.0 / 1  (0.0):   1%|          | 1/100 [00:01<02:39,  1.61s/it]

Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'


Average Metric: 0.0 / 2  (0.0):   2%|▏         | 2/100 [00:03<03:11,  1.96s/it]

Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'


Average Metric: 0.0 / 3  (0.0):   3%|▎         | 3/100 [00:05<03:08,  1.95s/it]

...

Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'


Average Metric: 2.5 / 99  (2.5):  99%|█████████▉| 99/100 [02:35<00:01,  1.79s/it]

Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'


Average Metric: 2.5 / 100  (2.5): 100%|██████████| 100/100 [02:37<00:00,  1.58s/it]
Compiled Question Decomposition Module Score: 2.5
Error analysis for uncompiled Question Decomposition module:
Total errors: 100

Sample of errors:
Original Question: Who is the spouse of the Green performer?
True Decomposition: 1. Green >> performer
2. #1 >> spouse
Predicted Decomposition: #1 >> Who is the Green performer?
#2 >> What is the name of the spouse of the identified Green performer?

Original Question: Who founded the company that distributed the film UHF?
True Decomposition: 1. UHF >> distributed by
2. #1 >> founded by
Predicted Decomposition: #1 >> What company distributed the film UHF?
#2 >> Who were the founders of Orion Pictures?
#3 >> When was Orion Pictures founded?
#4 >> What is the significance of Orion Pictures in the film industry?

Original Question: What administrative territorial entity is the owner of Ciudad Deportiva located?
True Decomposition: 1. Ciudad Deportiva >> owned b

Traceback (most recent call last):
  File "/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/litellm/caching.py", line 559, in add_cache
    self.cache.set_cache(cache_key, cached_data, **kwargs)
AttributeError: 'Cache' object has no attribute 'cache'
